In [2]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
from typing import Tuple, Union
import math
import mediapipe as mp
from keras import backend as K
import sys
import random
# from celluloid import Camera 
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from moviepy.editor import ImageSequenceClip
from IPython.display import display, Image

print(sys.executable)

c:\Users\Mgala\anaconda3\envs\lipnet\python.exe


In [3]:
# pixels
MARGIN = 0
ROW_SIZE = 0
FONT_SIZE = 1
FONT_THICKNESS = 1

# Input model Image size
IMAGE_SIZE = (50, 100)

batch_size = 1
mp_face_mesh =  mp.solutions.face_mesh.FaceMesh(max_num_faces=1,static_image_mode=True, min_detection_confidence=0.2)


## Functions

In [4]:
def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
    """Converts normalized values to pixel coordinates."""
    x_px = min(max(int(normalized_x * image_width), 0), image_width)
    y_px = min(max(int(normalized_y * image_height), 0), image_height)
    return x_px, y_px

In [5]:
def contrast_based_histogram_equalization_rgb(image):
    # # Convert the image to uint8 if not already
    # if image.dtype != np.uint8:
    #     image = (image * 255).astype(np.uint8)

    # Split channels
    channels = cv2.split(image)

    # Apply histogram equalization to each channel
    equalized_channels = [cv2.equalizeHist(channel) for channel in channels]

    # Merge channels
    equalized_image = cv2.merge(equalized_channels)

    return equalized_image

def clahe_equalization_rgb(image):
    # Convert the image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Split LAB image into L, A, B channels
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    clahe_l_channel = clahe.apply(l_channel)

    # Merge the enhanced L channel with the original A and B channels
    clahe_lab_image = cv2.merge((clahe_l_channel, a_channel, b_channel))

    # Convert the LAB image back to RGB
    clahe_rgb_image = cv2.cvtColor(clahe_lab_image, cv2.COLOR_LAB2RGB)

    return clahe_rgb_image

def scale_image_0_1(image):

    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))

    return normalized_image

def scale_image__1_1(image):

    normalized_image = 2 * (image - np.min(image, axis=(0, 1))) / (np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))) - 1
    normalized_image = np.clip(normalized_image, -1, 1)
    
    return normalized_image

def euclidean_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def inner_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

def display_gif(video):
    fig, ax = plt.subplots() # make it bigger
    camera = Camera(fig)# the camera gets our figure
    for img in os.listdir("NST/epochs"):
        img_obj = plt.imread(os.path.join("NST/epochs"), img) # reading
        ax.imshow(img_obj) # plotting
        camera.snap()
    animation = camera.animate()
    HTML(animation.to_html5_video())
    clip = ImageSequenceClip(list(scale_image_0_1(video.numpy())*255), fps=20)
    clip.write_gif('test.gif', fps=20)
    return Image('test.gif')


In [6]:
import cv2
import mediapipe as mp
import numpy as np

def use_camera():
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    average_angle = []
    while True:
        ret, frame = cap.read()

        if not ret:
            print("Broken image")
            continue

        # Convert to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get face mesh landmarks
        results = mp_face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                image_rows, image_cols, _ = frame.shape
                normalized_point = normalized_to_pixel_coordinates(1,1, image_cols, image_rows)

                left_lip = face_landmarks.landmark[0]
                right_lip = face_landmarks.landmark[17]
                left = face_landmarks.landmark[192]
                bottom= face_landmarks.landmark[199]
                top = face_landmarks.landmark[2]
                right = face_landmarks.landmark[416]

                bottom_px = normalized_to_pixel_coordinates(bottom.x, bottom.y, image_cols, image_rows)
                left_px = normalized_to_pixel_coordinates(left.x, left.y, image_cols, image_rows)
                top_px = normalized_to_pixel_coordinates(top.x, top.y, image_cols, image_rows)
                right_px = normalized_to_pixel_coordinates(right.x, right.y, image_cols, image_rows)
                left_lip = normalized_to_pixel_coordinates(left_lip.x, left_lip.y, image_cols, image_rows)
                right_lip = normalized_to_pixel_coordinates(right_lip.x, right_lip.y, image_cols, image_rows)


                midpoint = [(left_lip[0]+right_lip[0])//2,(left_lip[1]+right_lip[1])//2]

                # Calculate the differences in x and y coordinates
                dx = right_lip[0] - midpoint[0]
                dy = right_lip[1] - midpoint[1]
                
                # # Calculate the angle using arctan(dy/dx)
                angle_radians = math.atan2(dy, dx)
                
                # # Convert radians to degrees
                angle_degrees = math.degrees(angle_radians) - 90

                if len(average_angle) <6:
                    average_angle.append(angle_degrees)
                else:
                    average_angle.pop(0)
                    average_angle.append(angle_degrees)
                    
                if sum(average_angle) == 0:
                    angle_degrees = 0
                else:
                    angle_degrees = sum(average_angle) / len(average_angle)

                # # Rotate the image
                rotation_matrix = cv2.getRotationMatrix2D(midpoint, angle_degrees, 1.0)
                frame = cv2.warpAffine(frame, rotation_matrix, (normalized_point))

                # Convert rotated points back to list of tuples
                bottom_px , left_px , top_px , right_px = [(int(point[0][0]), int(point[0][1])) for point in cv2.transform(np.array([bottom_px , left_px , top_px , right_px]).reshape(-1, 1, 2).astype(np.float64), rotation_matrix)]

                if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                    continue

                frame = frame[
                        top_px[1] : bottom_px[1],
                        left_px[0] : right_px[0],
                        :,
                    ]
                
                if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                    continue

                frame = cv2.resize(
                        frame, (IMAGE_SIZE[1],IMAGE_SIZE[0]), interpolation=cv2.INTER_LANCZOS4
                    )
                
                frame = clahe_equalization_rgb(frame)

                mean = np.mean(frame, axis=(0, 1))  
                std = np.std(frame, axis=(0, 1))    
                frame = (frame - mean) / std
                cv2.imshow("frame", scale_image_0_1 (frame))
                break

        # Display the frame
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# use_camera()


In [7]:
MAX_LETTERS = 40

In [8]:
VOCAB ="ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩"
VOCAB

'ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩'

In [9]:
set1 = set(VOCAB.replace("\n",""))
set2 = set("ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩")
common_chars = set1.intersection(set2)
difference_set1 = set1.difference(set2)
difference_set2 = set2.difference(set1)

print(difference_set1, difference_set2)


set() set()


In [10]:
MAX_FRAMES = 138
MAX_FRAMES

138

In [11]:
vocab = [x for x in "ابتثجحخدذرزسشصضطظعغفقكلمنهويىء٠١٢٣٤٥٦٧٨٩ "]

In [12]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ى', 'ء', '٠', '١', '٢', '٣', '٤', '٥', '٦', '٧', '٨', '٩', ' '] (size =42)


In [13]:
def char_to_arab(alignments):
    sub_arrays = []
    current_sub_array = []
    for element in num_to_char(alignments).numpy():
        if element.decode("utf-8") == "":
            if current_sub_array:  # Add a non-empty sub-array
                sub_arrays.append(current_sub_array)
            current_sub_array = []  # Start a new sub-array
        else:
            current_sub_array.append(element.decode("utf-8"))

    if current_sub_array:  # Add the last sub-array if it's not empty
        sub_arrays.append(current_sub_array)

    for i in sub_arrays:
        print(''.join(i),end=" ")

In [ ]:
class ProduceExample(tf.keras.callbacks.Callback):
    def __init__(self, dataset) -> None:
        self.sample = train_data.as_numpy_iterator().next()

    def on_epoch_end(self, epoch, logs=None) -> None:
        data = self.sample
        yhat = self.model.predict(data[0],verbose=0)  # [75,75]

        decoded_200 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=200)[0][0].numpy()
        decoded_100 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=100)[0][0].numpy()
        decoded_50 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=50)[0][0].numpy()
        decoded_letter = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=char_to_num.vocabulary_size()+1)[0][0].numpy()
        decoded_40 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=40)[0][0].numpy()
        decoded_30 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=30)[0][0].numpy()
        decoded_20 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=20)[0][0].numpy()
        decoded_10 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=10)[0][0].numpy()
        decoded_5 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=5)[0][0].numpy()
        decoded_1 = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=False,beam_width=1)[0][0].numpy()
        decoded_1_greedy = tf.keras.backend.ctc_decode(yhat, [MAX_FRAMES], greedy=True)[0][0].numpy()

        for x in range(len(yhat)):
            print("\n\n")
            print("Original:",tf.strings.reduce_join(num_to_char(data[1][x])).numpy().decode("utf-8"))
            print("Prediction Dec 200:",tf.strings.reduce_join(num_to_char(decoded_200[x])).numpy().decode("utf-8"))
            print("Prediction Dec 100:",tf.strings.reduce_join(num_to_char(decoded_100[x])).numpy().decode("utf-8"))
            print("Prediction Dec 50:",tf.strings.reduce_join(num_to_char(decoded_50[x])).numpy().decode("utf-8"))
            print("Prediction Dec Voc:",tf.strings.reduce_join(num_to_char(decoded_letter[x])).numpy().decode("utf-8"))
            print("Prediction Dec 40:",tf.strings.reduce_join(num_to_char(decoded_40[x])).numpy().decode("utf-8"))
            print("Prediction Dec 30:",tf.strings.reduce_join(num_to_char(decoded_30[x])).numpy().decode("utf-8"))
            print("Prediction Dec 20:",tf.strings.reduce_join(num_to_char(decoded_20[x])).numpy().decode("utf-8"))
            print("Prediction Dec 10:",tf.strings.reduce_join(num_to_char(decoded_10[x])).numpy().decode("utf-8"))
            print("Prediction Dec 5:",tf.strings.reduce_join(num_to_char(decoded_5[x])).numpy().decode("utf-8"))
            print("Prediction Dec 1:",tf.strings.reduce_join(num_to_char(decoded_1[x])).numpy().decode("utf-8"))
            print("Prediction Dec Greedy:",tf.strings.reduce_join(num_to_char(decoded_1_greedy[x])).numpy().decode("utf-8"))
            print("\n\n")

In [23]:
def load_video(path: str) -> List[float]:
    # opens the video's path as a camera object
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # frames of the video
    frames = np.empty((MAX_FRAMES, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    average_angle = []
    idx = 0


    # loop over all frames in the video
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        # if _ == 0:
        #     continue
        try:
            ret, frame = cap.read()
            image_rows, image_cols, _ = frame.shape
            # convert to rgb
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # get the keypoints (x,y)s of each point of a face in the frame
            face_mesh_result = mp_face_mesh.process(frame)

            # loop over each face landmark ~ (nose (x,y) lips (x,y) and so one and a bounding box)
            if face_mesh_result.multi_face_landmarks:

                for face_landmarks in face_mesh_result.multi_face_landmarks:

                    normalized_point = normalized_to_pixel_coordinates(1,1, image_cols, image_rows)

                    left_lip = face_landmarks.landmark[0]
                    right_lip = face_landmarks.landmark[17]
                    left = face_landmarks.landmark[192]
                    bottom= face_landmarks.landmark[199]
                    top = face_landmarks.landmark[2]
                    right = face_landmarks.landmark[416]

                    bottom_px = normalized_to_pixel_coordinates(bottom.x, bottom.y, image_cols, image_rows)
                    left_px = normalized_to_pixel_coordinates(left.x, left.y, image_cols, image_rows)
                    top_px = normalized_to_pixel_coordinates(top.x, top.y, image_cols, image_rows)
                    right_px = normalized_to_pixel_coordinates(right.x, right.y, image_cols, image_rows)
                    left_lip = normalized_to_pixel_coordinates(left_lip.x, left_lip.y, image_cols, image_rows)
                    right_lip = normalized_to_pixel_coordinates(right_lip.x, right_lip.y, image_cols, image_rows)


                    midpoint = [(left_lip[0]+right_lip[0])//2,(left_lip[1]+right_lip[1])//2]

                    # Calculate the differences in x and y coordinates
                    dx = right_lip[0] - midpoint[0]
                    dy = right_lip[1] - midpoint[1]
                    
                    # # Calculate the angle using arctan(dy/dx)
                    angle_radians = math.atan2(dy, dx)
                    
                    # # Convert radians to degrees
                    angle_degrees = math.degrees(angle_radians) - 90

                    if len(average_angle) <6:
                        average_angle.append(angle_degrees)
                    else:
                        average_angle.pop(0)
                        average_angle.append(angle_degrees)
                        
                    if sum(average_angle) == 0:
                        angle_degrees = 0
                    else:
                        angle_degrees = sum(average_angle) / len(average_angle)

                    # Rotate the image
                    rotation_matrix = cv2.getRotationMatrix2D(midpoint, angle_degrees, 1.0)
                    frame = cv2.warpAffine(frame, rotation_matrix, (normalized_point))

                    # Convert rotated points back to list of tuples
                    bottom_px , left_px , top_px , right_px = [(int(point[0][0]), int(point[0][1])) for point in cv2.transform(np.array([bottom_px , left_px , top_px , right_px]).reshape(-1, 1, 2).astype(np.float64), rotation_matrix)]

                    if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                        continue

                    frame = frame[
                            top_px[1] : bottom_px[1],
                            left_px[0] : right_px[0],
                            :,
                        ]
                    
                    if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                        continue

                    frame = cv2.resize(
                            frame, (IMAGE_SIZE[1],IMAGE_SIZE[0]), interpolation=cv2.INTER_LANCZOS4
                        )
                    
                    frame = clahe_equalization_rgb(frame)

                    mean = np.mean(frame, axis=(0, 1))  
                    std = np.std(frame, axis=(0, 1))    
                    frame = (frame - mean) / std

                    frames[idx] = frame

                    idx += 1
                    break

        except Exception as e:
                # Handle the case where the landmarks are not found
            print(e)
            return None

    cap.release()
    cv2.waitKey(0)

    # normalize on the scale of the video
    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    frames = tf.cast((frames - mean), tf.float32) / std

    if idx < MAX_FRAMES:
        for i in range(idx, MAX_FRAMES):
            frames = tf.tensor_scatter_nd_update(frames, [[i]], tf.zeros((1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3), dtype=tf.float32))

    # frames = tf.cast(tf.convert_to_tensor(frames),tf.float32)
    
    return frames.numpy()


In [67]:
def load_alignments(path:str) -> List[str]: 
    with open(path, 'r') as f: 
        lines = f.readlines() 
    tokens = []
    for line in lines:
        line = line.strip("\n")
        tokens = [*tokens,' ',line]

    return char_to_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [24]:
from tensorflow.keras.models import Sequential ,model_from_json
from tensorflow.keras.layers import Conv3D,GRU, Dense, Dropout, Bidirectional, MaxPool3D, Activation, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import keras


In [86]:
model = Sequential()
model.add(Conv3D(32,(1, 5, 5),strides=(1,2,2), input_shape=(MAX_FRAMES, IMAGE_SIZE[0],IMAGE_SIZE[1], 3), padding="same"))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(64,(1, 5, 5), padding="same"))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(Conv3D(96,(1, 3, 3), padding="same"))
model.add(Activation('relu'))
model.add(MaxPool3D((1,2,2)))

model.add(TimeDistributed(Flatten()))

model.add(Bidirectional(GRU(256,kernel_initializer=keras.initializers.Orthogonal(), return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(256,kernel_initializer=keras.initializers.Orthogonal(), return_sequences=True)))
model.add(Dropout(0.5))

model.add(Dense(char_to_num.vocabulary_size()+1, kernel_initializer='he_normal', activation='softmax'))
model.load_weights(r"E:/37/test1.weights.hdf5")

In [89]:
import glob

for idx,i in enumerate(glob.glob(r"E:/Video Links Dataset/filtered/videos_25fps/*.mp4")):  
    i = i.replace("\\","/") 
    x = load_video(i.replace("\\","/"))
    model_pred = model.predict([x.reshape((1,*x.shape))])
    decoded_pred = tf.keras.backend.ctc_decode(model_pred, [MAX_FRAMES], greedy=False,beam_width=200)[0][0].numpy()
    decoded_text = tf.strings.reduce_join(num_to_char(decoded_pred[0])).numpy().decode("utf-8")
    print(decoded_text)
    print(char_to_arab(load_alignments(fr"E:/Video Links Dataset/filtered/alignments/{i.split('/')[-1].split('.')[0]}.align").numpy()))
    print("\n\n\n")
    if idx == 15:
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
ده كانا مي في الشغل الايه ما بي هو
ده كان زميلي في الشغل لغايه ما بعني هو None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
تني
اتنين None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
وزير الصحه بقى فضي يقال اي حد كده وانت
وزير الصحه بقى فاضي يقابل اي حد كده وانت None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
الافواه واضطاده الصحين المعارضين
الافواه واضطهاد الصحفيين المعارضين None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
الو بت الي
الو نصبت الاستيج None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
خلص ايه ديه يرحا الع طب عره ما كان
خلاص ايه ده يا رحاب الع طب عمره ما كان None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
الي الار بتعله لو طفل ما اتاصا او
اللي الدار بتعمله لو طفل مثلا اتصاب او None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
ويا بخت كل طفل هن بام زي حضتك بس كان
ويا بخت كل طفل هنا بام زي حضرتك بس كان None




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
ده مو في الدولا بتعه فحبيتا عرف
ده موجود في الدولاب بتاعها فحبيت اعرف None




1/1 ━━━━━━━━━━━

In [61]:
decoded_pred

array([[27, 23,  1, 41, 24,  1, 41,  1, 23, 28, 41, 18, 23, 18, 41, 18,
        23,  1, 41,  1, 18, 41,  1, 25, 41,  1,  2,  0, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int64)

In [62]:
decoded_text = tf.strings.reduce_join(num_to_char(decoded_pred[0])).numpy().decode("utf-8")
decoded_text

'ولا ما الي علع علا اع ان اب'

In [76]:
char_to_arab(load_alignments(r"E:/data/me/alignments/18 Galal.align").numpy())

بعد اذنك انا عايز اعدى 

In [65]:
import tensorflow as tf
from tensorflow.keras import backend as K


class CERMetric(tf.keras.metrics.Metric):
    """
    A custom Keras metric to compute the Character Error Rate
    """
    def __init__(self, name='CER_metric', **kwargs):
        super(CERMetric, self).__init__(name=name, **kwargs)
        self.cer_accumulator = self.add_weight(name="total_cer", initializer="zeros")
        self.counter = self.add_weight(name="cer_count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):

        input_shape = K.shape(y_pred)
        input_length = tf.ones(shape=input_shape[0], dtype=tf.int64) * K.cast(input_shape[1], tf.int64)

        decode, log = K.ctc_decode(y_pred,
                                    input_length,
                                    greedy=True)

        decode = K.ctc_label_dense_to_sparse(decode[0], K.cast(input_length, tf.int32))
        y_true_sparse = K.ctc_label_dense_to_sparse(y_true, K.cast(input_length, tf.int32))

        decode = tf.sparse.retain(decode, tf.not_equal(decode.values, -1))
        distance = tf.edit_distance(decode, y_true_sparse, normalize=True)

        self.cer_accumulator.assign_add(tf.reduce_sum(distance))
        self.counter.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return tf.math.divide_no_nan(self.cer_accumulator, self.counter)

    def reset_states(self):
        self.cer_accumulator.assign(0.0)
        self.counter.assign(0.0)

In [ ]:
def Calc_WER(y_true, y_pred):
    for word in y_true.split():
        

In [74]:
wer = WERMetric(
    load_alignments(r"E:/data/me/alignments/18 Galal.align").numpy(),
                decoded_pred[0])


TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
wer.update_state()

In [ ]:
بعد
اذنك
انا
عايز
اعدى